# BCDP CORDEX Example
Here we will do a quick walkthrough of using BCDP to process some regional climate simulations from the Coordinate Regional Downscaling Experiment

In [1]:
import os
import glob
import numpy as np
import holoviews as hv
import hvplot.xarray
import bcdp

Create a file conventions template. Here we only need the model and variable names so the rest of the file template is filled in by wildcards.

In [2]:
project = 'CORDEX-Africa'
template = '*_{model}_*_{variable}.nc'
bcdp.build_extractor(project, template, name_field='model', index=[1, 6])

bcdp.extractors.build_extractor_from_params.<locals>.GenericExtractor

Load the data. Because we have loaded the template, the loader now knows exactly how to extract the required informations from the filenames.

In [3]:
paths = os.path.join(os.path.expanduser('~'), 'data/CORDEX_Africa/*clt*')
ens = bcdp.load_local(paths=paths, project=project)
print(ens.size)

297.91MB


The loader returns an `Ensemble` object, which is essentially a collection of datasets and applies preprocessing operations to each of them. Here we will regrid the data to a coarser (0.88 degree) grid using bilinear interpolation, and consider only the winter months (DJF).

In [4]:
output_grid = bcdp.utils.grid_from_res((0.88, 0.88), ens.overlap)
ens = ens.homogenize(freq='Y', season='DJF', backend='scipy', method='linear',
                     output_grid=output_grid, clean=False, reuse_weights=True)

/Users/goodman/opt/lib/python3.6/site-packages/xarray/core/resample.py:174: FutureWarning: This DataArray contains multi-dimensional coordinates. In the future, the dimension order of these coordinates will be restored as well unless you specify restore_coord_dims=False.
  super().__init__(*args, **kwargs)


Now that the underlying datastructures are homogeneous (same grid and time step), we can convert it to an xarray dataarray which has dimensions (names, time, lat, lon).

In [5]:
da = ens.bundle('CORDEX').add_mean('CORDEX').first

We can easily visualize the annual climatology with xarray's built-in plotting methods.

In [6]:
plot = da.hvplot.contourf(groupby='time', col='names', x='x', y='y')
hv.Layout([i.opts(width=230, height=230, title=title) for (n,i), title in zip(plot.items(), da.names.values)]).cols(3)

:Layout
   .DynamicMap.I    :DynamicMap   [time]
      :Polygons   [x,y]   (CORDEX)
   .DynamicMap.II   :DynamicMap   [time]
      :Polygons   [x,y]   (CORDEX)
   .DynamicMap.III  :DynamicMap   [time]
      :Polygons   [x,y]   (CORDEX)
   .DynamicMap.IV   :DynamicMap   [time]
      :Polygons   [x,y]   (CORDEX)
   .DynamicMap.V    :DynamicMap   [time]
      :Polygons   [x,y]   (CORDEX)
   .DynamicMap.VI   :DynamicMap   [time]
      :Polygons   [x,y]   (CORDEX)
   .DynamicMap.VII  :DynamicMap   [time]
      :Polygons   [x,y]   (CORDEX)
   .DynamicMap.VIII :DynamicMap   [time]
      :Polygons   [x,y]   (CORDEX)
   .DynamicMap.IX   :DynamicMap   [time]
      :Polygons   [x,y]   (CORDEX)